In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import classification_report

# Define data directories
train_dir = 'Path Of Traning Folder'
validation_dir = 'Path Of Test Folder'
test_dir = 'Path Of Validation Folder'

# Define data generators
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Set batch size and image dimensions
batch_size = 32
image_height = 64
image_width = 64

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='binary'
)

# Build a DNN model
model = Sequential([
    Flatten(input_shape=(image_height, image_width, 3)),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Adding dropout for regularization
    Dense(64, activation='relu'),
    Dropout(0.5),  # Adding dropout for regularization
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

# Train the model
epochs = 1  # Increase the number of epochs for better training
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='binary'
)

test_loss, test_acc, test_precision, test_recall = model.evaluate(test_generator)

# Predict classes for the test set
y_pred = model.predict(test_generator)
y_pred_classes = (y_pred > 0.5).astype(int)

# Calculate F1-score, precision, and recall
report = classification_report(test_generator.classes, y_pred_classes)

print(f"Test accuracy: {test_acc}")
print(f"Test precision: {test_precision}")
print(f"Test recall: {test_recall}")
print("Classification Report:\n", report)